In [1]:
import pandas as pd
from datetime import datetime

In [2]:
# file paths for HTML data
january_file = '/Users/spencergreen/NBA-Prediction-Model/Data/Updated_Standings/NBA_2024_games-january_updated.html'
february_file = '/Users/spencergreen/NBA-Prediction-Model/Data/Updated_Standings/NBA_2024_games-february.html'
march_file = '/Users/spencergreen/NBA-Prediction-Model/Data/Updated_Standings/NBA_2024_games-march.html'
april_file = '/Users/spencergreen/NBA-Prediction-Model/Data/Updated_Standings/NBA_2024_games-april.html'


In [3]:
# reading the HTML files
january_df = pd.read_html(january_file)[0]
february_df = pd.read_html(february_file)[0]
march_df = pd.read_html(march_file)[0]
april_df = pd.read_html(april_file)[0]

In [4]:
# combining the DataFrames
combined_fixtures_df = pd.concat([january_df, february_df, march_df, april_df], ignore_index=True)

In [5]:
combined_fixtures_df.head()

,Date,Start (ET),Visitor/Neutral,PTS,Home/Neutral,PTS.1,Unnamed: 6,Unnamed: 7,Attend.,Arena,Notes
0,"Wed, Jan 24, 2024",7:00p,Charlotte Hornets,NaN,Detroit Pistons,NaN,NaN,NaN,NaN,Little Caesars Arena,NaN
1,"Wed, Jan 24, 2024",7:00p,Minnesota Timberwolves,NaN,Washington Wizards,NaN,NaN,NaN,NaN,Capital One Arena,NaN
2,"Wed, Jan 24, 2024",7:30p,Memphis Grizzlies,NaN,Miami Heat,NaN,NaN,NaN,NaN,Kaseya Center,NaN
3,"Wed, Jan 24, 2024",8:00p,Portland Trail Blazers,NaN,Houston Rockets,NaN,NaN,NaN,NaN,Toyota Center,NaN
4,"Wed, Jan 24, 2024",8:00p,Cleveland Cavaliers,NaN,Milwaukee Bucks,NaN,NaN,NaN,NaN,Fiserv Forum,NaN


In [6]:
# convert date string to a proper datetime format
def convert_date(date_str):
    try:
        return datetime.strptime(date_str, "%a, %b %d, %Y")
    except ValueError:
            return None 
    
# apply the adjusted function to convert dates/drop rows with invalid dates
combined_fixtures_df['date'] = combined_fixtures_df['Date'].apply(convert_date)
combined_fixtures_df.dropna(subset=['date'], inplace=True)

In [7]:
# create a new DataFrame for home/away games
home_games = combined_fixtures_df[['date', 'Home/Neutral']].rename(columns={'Home/Neutral': 'team'})
home_games['home'] = True


away_games = combined_fixtures_df[['date', 'Visitor/Neutral']].rename(columns={'Visitor/Neutral': 'team'})
away_games['home'] = False

# combine home and away games into one DataFrame
all_games = pd.concat([home_games, away_games], ignore_index=True).sort_values(by='date')


In [8]:
# Add columns for 'home_next' date_next'
all_games['home_next'] = all_games.groupby('team')['home'].shift(-1)
all_games['date_next'] = all_games.groupby('team')['date'].shift(-1)

In [9]:
all_games

,date,team,home,home_next,date_next
0,2024-01-24,Detroit Pistons,True,True,2024-01-27
1,2024-01-24,Washington Wizards,True,True,2024-01-25
2,2024-01-24,Miami Heat,True,True,2024-01-25
3,2024-01-24,Houston Rockets,True,False,2024-01-26
4,2024-01-24,Milwaukee Bucks,True,True,2024-01-26
...,...,...,...,...,...
569,2024-04-14,Indiana Pacers,True,NaN,NaT
568,2024-04-14,Cleveland Cavaliers,True,NaN,NaT
567,2024-04-14,Boston Celtics,True,NaN,NaT
1162,2024-04-14,Portland Trail Blazers,False,NaN,NaT


In [10]:
# logic for 'team_opp_next'
# for each game, the opponent will be the 'Visitor/Neutral' if the team is 'Home/Neutral', and vice versa
all_games['team_opp'] = all_games.apply(
    lambda row: combined_fixtures_df.loc[
        (combined_fixtures_df['date'] == row['date']) & 
        (combined_fixtures_df['Home/Neutral'] == row['team']), 
        'Visitor/Neutral'
    ].iloc[0] 
    if row['home'] 
    else combined_fixtures_df.loc[
        (combined_fixtures_df['date'] == row['date']) & 
        (combined_fixtures_df['Visitor/Neutral'] == row['team']), 
        'Home/Neutral'
    ].iloc[0], 
    axis=1
)

# Update 'team_opp_next' to show the next opponent
all_games['team_opp_next'] = all_games.groupby('team')['team_opp'].shift(-1)

In [11]:
all_games

,date,team,home,home_next,date_next,team_opp,team_opp_next
0,2024-01-24,Detroit Pistons,True,True,2024-01-27,Charlotte Hornets,Washington Wizards
1,2024-01-24,Washington Wizards,True,True,2024-01-25,Minnesota Timberwolves,Utah Jazz
2,2024-01-24,Miami Heat,True,True,2024-01-25,Memphis Grizzlies,Boston Celtics
3,2024-01-24,Houston Rockets,True,False,2024-01-26,Portland Trail Blazers,Charlotte Hornets
4,2024-01-24,Milwaukee Bucks,True,True,2024-01-26,Cleveland Cavaliers,Cleveland Cavaliers
...,...,...,...,...,...,...,...
569,2024-04-14,Indiana Pacers,True,NaN,NaT,Atlanta Hawks,NaN
568,2024-04-14,Cleveland Cavaliers,True,NaN,NaT,Charlotte Hornets,NaN
567,2024-04-14,Boston Celtics,True,NaN,NaT,Washington Wizards,NaN
1162,2024-04-14,Portland Trail Blazers,False,NaN,NaT,Sacramento Kings,NaN


In [12]:
# mapping of full team names to their abbreviations
team_abbreviations = {
    "Atlanta Hawks": "ATL",
    "Boston Celtics": "BOS",
    "Brooklyn Nets": "BKN",
    "Charlotte Hornets": "CHA",
    "Chicago Bulls": "CHI",
    "Cleveland Cavaliers": "CLE",
    "Dallas Mavericks": "DAL",
    "Denver Nuggets": "DEN",
    "Detroit Pistons": "DET",
    "Golden State Warriors": "GSW",
    "Houston Rockets": "HOU",
    "Indiana Pacers": "IND",
    "Los Angeles Clippers": "LAC",
    "Los Angeles Lakers": "LAL",
    "Memphis Grizzlies": "MEM",
    "Miami Heat": "MIA",
    "Milwaukee Bucks": "MIL",
    "Minnesota Timberwolves": "MIN",
    "New Orleans Pelicans": "NOP",
    "New York Knicks": "NYK",
    "Oklahoma City Thunder": "OKC",
    "Orlando Magic": "ORL",
    "Philadelphia 76ers": "PHI",
    "Phoenix Suns": "PHX",
    "Portland Trail Blazers": "POR",
    "Sacramento Kings": "SAC",
    "San Antonio Spurs": "SAS",
    "Toronto Raptors": "TOR",
    "Utah Jazz": "UTA",
    "Washington Wizards": "WAS"
}



In [13]:
# replace the team names with abbreviations
all_games['team'] = all_games['team'].replace(team_abbreviations)
all_games['team_opp_next'] = all_games['team_opp_next'].replace(team_abbreviations)
all_games['team_opp'] = all_games['team_opp'].replace(team_abbreviations)

In [14]:
all_games

,date,team,home,home_next,date_next,team_opp,team_opp_next
0,2024-01-24,DET,True,True,2024-01-27,CHA,WAS
1,2024-01-24,WAS,True,True,2024-01-25,MIN,UTA
2,2024-01-24,MIA,True,True,2024-01-25,MEM,BOS
3,2024-01-24,HOU,True,False,2024-01-26,POR,CHA
4,2024-01-24,MIL,True,True,2024-01-26,CLE,CLE
...,...,...,...,...,...,...,...
569,2024-04-14,IND,True,NaN,NaT,ATL,NaN
568,2024-04-14,CLE,True,NaN,NaT,CHA,NaN
567,2024-04-14,BOS,True,NaN,NaT,WAS,NaN
1162,2024-04-14,POR,False,NaN,NaT,SAC,NaN


In [15]:
# function to convert True/False to 1/0, and keep NaN as is
convert_bool = lambda x: 1 if x == True else (0 if x == False else x)

# Applying to 'Home' and 'home_next' columns
all_games['home'] = all_games['home'].apply(convert_bool)
all_games['home_next'] = all_games['home_next'].apply(convert_bool)

In [16]:
all_games

,date,team,home,home_next,date_next,team_opp,team_opp_next
0,2024-01-24,DET,1,1.0,2024-01-27,CHA,WAS
1,2024-01-24,WAS,1,1.0,2024-01-25,MIN,UTA
2,2024-01-24,MIA,1,1.0,2024-01-25,MEM,BOS
3,2024-01-24,HOU,1,0.0,2024-01-26,POR,CHA
4,2024-01-24,MIL,1,1.0,2024-01-26,CLE,CLE
...,...,...,...,...,...,...,...
569,2024-04-14,IND,1,NaN,NaT,ATL,NaN
568,2024-04-14,CLE,1,NaN,NaT,CHA,NaN
567,2024-04-14,BOS,1,NaN,NaT,WAS,NaN
1162,2024-04-14,POR,0,NaN,NaT,SAC,NaN


In [17]:
all_games["season"] = 2024

In [18]:
all_games

,date,team,home,home_next,date_next,team_opp,team_opp_next,season
0,2024-01-24,DET,1,1.0,2024-01-27,CHA,WAS,2024
1,2024-01-24,WAS,1,1.0,2024-01-25,MIN,UTA,2024
2,2024-01-24,MIA,1,1.0,2024-01-25,MEM,BOS,2024
3,2024-01-24,HOU,1,0.0,2024-01-26,POR,CHA,2024
4,2024-01-24,MIL,1,1.0,2024-01-26,CLE,CLE,2024
...,...,...,...,...,...,...,...,...
569,2024-04-14,IND,1,NaN,NaT,ATL,NaN,2024
568,2024-04-14,CLE,1,NaN,NaT,CHA,NaN,2024
567,2024-04-14,BOS,1,NaN,NaT,WAS,NaN,2024
1162,2024-04-14,POR,0,NaN,NaT,SAC,NaN,2024


In [19]:
all_games.to_csv('nba_updated_fixtures.csv') 